# Regression and Other Stories: ChileSchools

Code and figures for ChileSchools example. See Chapter 21 in Regression and Other Stories.

Data are from

Chay, K. Y., McEwan, P. J., and Urquiola, M. (2005). The central role of noise in evaluating interventions that use test scores to rank schools. American Economic Review 95, 1237–1258.

----

#### Load Packages

In [1]:
using DataFrames, StatsPlots, CSV, HTTP
using Distributions, Turing, MCMCChains

#### Load data

The outcomes in these analyses are the gain scores between 88 and 92.

In [2]:
data = "https://raw.githubusercontent.com/avehtari/ROS-Examples/master/ChileSchools/data/chile.csv"
chile  = CSV.File(HTTP.get(data).body, delim=",") |> DataFrame
first(chile, 6)

,p90,cmb_regn,urban88,rule2,cutoff,cutoff_cmb,eligible,read92
,Int64,String,Int64,Float64,Float64,Float64,Int64,Float64
1,0,"Regions 2,5,10",1,3.415,49.4,49.4,0,57.0
2,0,"Regions 6,8",1,31.815,43.4,43.4,0,85.515
3,0,"Regions 6,8",1,4.445,43.4,43.4,0,51.971
4,0,Region 13,1,16.8,46.4,46.4,0,66.374
5,0,"Regions 2,5,10",1,0.0949989,49.4,49.4,0,52.5
6,1,Region 13,1,-2.03,46.4,46.4,1,55.333


### Fit models

In [4]:
# Model 1: No interaction terms
@model function model_1(eligible, rule2, read92)
    σ ~ Exponential(10)
    
    α ~ Normal(0, 100)
    β_e ~ Normal(0, 10)
    β_r ~ Normal(0, 10)
    
    μ = α .+ eligible * β_e .+ rule2 * β_r
    read92 ~ MvNormal(μ, σ)
end

# Model 2: Add interaction between eligible and rule2
@model function model_2(eligible, rule2, read92)
    σ ~ Exponential(10)
    
    α ~ Normal(0, 100)
    β_e ~ Normal(0, 10)
    β_r ~ Normal(0, 10)
    β_er ~ Normal(0,10)
    
    μ = α .+ eligible * β_e .+ rule2 * β_r + eligible * rule2 * β_er
    read92 ~ MvNormal(μ, σ)
end

# Model 3: Add read88 and math88
@model function model_3(eligible, rule2, read88, math88, read92)
    σ ~ Exponential(10)
    
    α ~ Normal(0, 100)
    β_eligible ~ Normal(0, 10)
    β_rule2 ~ Normal(0, 10)
    β_read88 ~ Normal(0,10)
    β_math88 ~ Normal(0,10)
    
    μ = α .+ eligible * β_e .+ rule2 * β_r .+ read88 * β_read88 .+ math88 * β_math88 
    read92 ~ MvNormal(μ, σ)
end;

In [10]:
model_1a = model_1(chile.eligible, chile.rule2, chile.read92)
fit_1a = sample(model_1a, NUTS(), 2000)

chile_subset = filter(:rule2 => rule2 -> abs(rule2) < 5, chile)
model_1b = model_1(chile_subset.eligible, chile_subset.rule2, chile_subset.read92)
fit_1b = sample(model_1b, NUTS(), 2000)

model_2b = model_2(chile.eligible, chile.rule2, chile.read92)
fit_2b = sample(model_2b, NUTS(), 2000)

model_3a = model_3(chile.eligible, chile.rule2, chile.read88, chile.math88, chile.read92)
fit_3a = sample(model_3a, NUTS(), 2000)

model_3b = model_3(
    chile_subset.eligible, 
    chile_subset.rule2, 
    chile_subset.read88, 
    chile_subset.math88, 
    chile_subset.read92)
fit_3b = sample(model_3b, NUTS(), 2000)


┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)

LoadError: MethodError: no method matching *(::Vector{Int64}, ::Vector{Float64})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  *([91m::StridedMatrix{T}[39m, ::StridedVector{S}) where {T<:Union{Float32, Float64, ComplexF32, ComplexF64}, S<:Real} at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\matmul.jl:44
[0m  *(::StridedVecOrMat{T} where T, [91m::LinearAlgebra.Adjoint{var"#s832", var"#s831"} where {var"#s832", var"#s831"<:LinearAlgebra.LQPackedQ}[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\lq.jl:254
[0m  ...